In [ ]:
!pip install trl

In [2]:
from IPython.display import Image

In [3]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['WANDB_DISABLED'] = 'true'
os.environ['HF_HOME'] = '/home/samtang/.hf/'

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

## collate_fn (dataloader)

In [6]:
Image(url='https://lukesalamone.github.io/img/torch_collate_fn.png', width=400)

In [9]:
dataset = load_dataset('lucasmccabe-lmi/CodeAlpaca-20k', split='train')

In [10]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 20022
})

In [9]:
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

## format function

In [21]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question:\n{example['instruction'][i]}\n ### Answer:<|end_header_id|>\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts


In [83]:
output_texts = formatting_prompts_func(dataset[:2])

In [84]:
print(dataset[0], '\n')
print(output_texts[0])

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.', 'input': '', 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'} 

### Question:
Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.
 ### Answer:<|end_header_id|>
def f(x):
    """
    Takes a specific input and produces a specific output using any mathematical operators
    """
    return x**2 + 3*x


## finetune facebook opt-350m

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [15]:
tokenizer

GPT2TokenizerFast(name_or_path='facebook/opt-350m', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [85]:
tokenizer.chat_template

In [86]:
response_template = "<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [87]:
dataset[:2]

{'instruction': ['Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
  'Generate a unique 8 character string that contains a lowercase letter, an uppercase letter, a numerical digit, and a special character. Write corresponding code in Python.'],
 'input': ['', ''],
 'output': ['def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x',
  "import string\nimport random\n\ndef random_password_string():\n    characters = string.ascii_letters + string.digits + string.punctuation\n    password = ''.join(random.sample(characters, 8))\n    return password\n\nif __name__ == '__main__':\n    print(random_password_string())"]}

In [88]:
tokenized_data = [tokenizer.encode(s) for s in formatting_prompts_func(dataset[:2])]

In [89]:
from torch.utils.data import DataLoader
# print(tokenizer.decode(tokenized_data[0]))

dataloader = DataLoader(tokenized_data, collate_fn=collator, batch_size=1)

In [90]:
row = next(iter(dataloader))
row

{'input_ids': tensor([[    2, 48134, 15680,    35, 50118, 44758,    10,  5043,    14,  1239,
             10,  2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143,
          30412,  5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118,
          22560, 31652,    35, 41552, 15483,  1397,  1215, 24419,  1215,   808,
          15483, 15698, 50118,  9232,   856,  1640,  1178,  3256, 50118,  1437,
           1437,  1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,
           8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990,
          50118,  1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,
           3023, 12606,   176,  2055,   155,  3226,  1178]]),
 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -10

In [71]:
print(tokenizer.decode([2, 48134, 15680,    35, 50118, 44758,    10,  5043,    14,  1239,
             10,  2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143,
          30412,  5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118,
          22560, 31652,    35, 50118,  9232,   856,  1640,  1178,  3256, 50118,
           1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,
           2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
           5990, 50118,  1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,
            671,  3023, 12606,   176,  2055,   155,  3226,  1178]))

</s>### Question:
Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.
 ### Answer:
def f(x):
    """
    Takes a specific input and produces a specific output using any mathematical operators
    """
    return x**2 + 3*x


In [92]:
print(tokenizer.decode([50118,  9232,   856,  1640,  1178,  3256, 50118,  1437,
           1437,  1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,
           8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990,
          50118,  1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,
           3023, 12606,   176,  2055,   155,  3226,  1178]))


def f(x):
    """
    Takes a specific input and produces a specific output using any mathematical operators
    """
    return x**2 + 3*x


#### 1. SFTTrainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=SFTConfig(output_dir='/tmp', report_to='wandb'),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 20022/20022 [00:00<00:00, 28582.15 examples/s]


#### 2. train()

In [94]:
trainer.train()

  7%|▋         | 500/7509 [01:34<23:41,  4.93it/s]

{'loss': 1.4623, 'grad_norm': 8.518695831298828, 'learning_rate': 1.8668264748967905e-05, 'epoch': 0.2}


 13%|█▎        | 1000/7509 [03:09<20:13,  5.36it/s] 

{'loss': 1.2277, 'grad_norm': 8.79334545135498, 'learning_rate': 1.733652949793581e-05, 'epoch': 0.4}


 20%|█▉        | 1500/7509 [04:41<26:59,  3.71it/s]  

...

100%|█████████▉| 7500/7509 [23:32<00:01,  6.45it/s]

{'loss': 0.6701, 'grad_norm': 6.4568586349487305, 'learning_rate': 2.3971234518577706e-08, 'epoch': 3.0}


100%|██████████| 7509/7509 [23:38<00:00,  5.29it/s]

{'train_runtime': 1418.8425, 'train_samples_per_second': 42.335, 'train_steps_per_second': 5.292, 'train_loss': 0.9089754720330476, 'epoch': 3.0}


TrainOutput(global_step=7509, training_loss=0.9089754720330476, metrics={'train_runtime': 1418.8425, 'train_samples_per_second': 42.335, 'train_steps_per_second': 5.292, 'total_flos': 3.0378389083324416e+16, 'train_loss': 0.9089754720330476, 'epoch': 3.0})

In [107]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [108]:
from transformers import pipeline

#### 3. test finetune effect

In [109]:
pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [120]:
ans = pipeline('Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.', max_length=512)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [121]:
print(ans[0]['generated_text'])

Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.

def get_output(x):
    return x**2 + x**2

print(get_output(5))

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output:
5

Output

## finetune qwen2.5-7b

#### 1. load model and tokenizer

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning:

#### 2. evaluate origin generate text

In [3]:
from datasets import load_dataset
dataset = load_dataset('lucasmccabe-lmi/CodeAlpaca-20k', split='train')
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

In [4]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (rotary_emb): Qwen2RotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): Qwen2MLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3584, out_features=152064, bias=False)
)

In [ ]:
# pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)
# generator = pipeline('text-generation', model, tokenizer, device_map='auto')
# result = generator({"question":"Hello, how are you?"}, max_length=50)
# result
import torch
# cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_text = dataset['instruction'][0]
inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
# cpu time >2m40s

OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacity of 23.61 GiB of which 177.44 MiB is free. Including non-PyTorch memory, this process has 22.19 GiB memory in use. Of the allocated memory 21.42 GiB is allocated by PyTorch, and 323.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.to(device)
# Generate text
outputs = model.generate(inputs["input_ids"], max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [15]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [25]:
sftconfig = SFTConfig(
    output_dir='./tmp',
    warmup_ratio=0.1,
    # report_to='wandb',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


#### 3. get peft model

In [26]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [13]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

In [14]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (rotary_emb): Qwen2RotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
                (quant_linear_modu

In [27]:
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 1,170,939,392 || trainable%: 6.8953


In [44]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

In [70]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": 'You are a useful assistant to generate python function by user instruction!'},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": "xldjalskdajlsxx"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False
)
print(text)

<|im_start|>system
You are a useful assistant to generate python function by user instruction!<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
xldjalskdajlsxx<|im_end|>



In [61]:
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

In [71]:
def prepare_dataset(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        messages = [
            {"role": "system", "content": 'You are a useful assistant to generate python function by user instruction!'},
            {"role": "user", "content": example['instruction'][i]},
            {"role": "assistant", "content": example['output'][i]}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        output_texts.append(text)
    return output_texts

print(prepare_dataset(dataset[:1]))

['<|im_start|>system\nYou are a useful assistant to generate python function by user instruction!<|im_end|>\n<|im_start|>user\nCreate a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.<|im_end|>\n<|im_start|>assistant\ndef f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x<|im_end|>\n']


In [74]:
from torch.utils.data import DataLoader
collator = DataCollatorForCompletionOnlyLM(response_template='<|im_start|>assistant', tokenizer=tokenizer)

tokenized_data = [tokenizer.encode(s) for s in prepare_dataset(dataset[:1])]
dataloader = DataLoader(tokenized_data, collate_fn=collator, batch_size=1)
row = next(iter(dataloader))
row

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,   5390,  17847,    311,
            6923,  10135,    729,    553,   1196,   7600,      0, 151645,    198,
          151644,    872,    198,   4021,    264,    729,    429,   4990,    264,
            3151,   1946,    323,  18644,    264,   3151,   2550,   1667,    894,
           35972,  19624,     13,   9645,  12159,   2038,    304,  13027,     13,
          151645,    198, 151644,  77091,    198,    750,    282,   2075,    982,
             262,   3190,    262,  37607,    264,   3151,   1946,    323,  18644,
             264,   3151,   2550,   1667,    894,  35972,  19624,    198,    262,
            3190,    262,    470,    856,    334,     17,    488,    220,     18,
           18481, 151645,    198]]),
 'labels': tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
            -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
            -100,   -100,   -100,   -1

In [77]:
print(tokenizer.decode([198,    750,    282,   2075,    982,
             262,   3190,    262,  37607,    264,   3151,   1946,    323,  18644,
             264,   3151,   2550,   1667,    894,  35972,  19624,    198,    262,
            3190,    262,    470,    856,    334,     17,    488,    220,     18,
           18481, 151645,    198]))


def f(x):
    """
    Takes a specific input and produces a specific output using any mathematical operators
    """
    return x**2 + 3*x<|im_end|>



In [79]:
trainer = SFTTrainer(
    model=model,
    args=sftconfig,
    train_dataset=dataset,
    formatting_func=prepare_dataset,
    tokenizer=tokenizer,
    data_collator=collator,
)

Map: 100%|██████████| 20022/20022 [00:01<00:00, 14958.20 examples/s]


In [80]:
trainer.train()             

  0%|          | 0/30033 [00:00<?, ?it/s]/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  2%|▏         | 500/30033 [04:23<4:04:48,  2.01it/s]

{'loss': 1.4866, 'grad_norm': 2.6288650035858154, 'learning_rate': 3.3288948069241015e-06, 'epoch': 0.05}


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  3%|▎         | 1000/30033 [08:49<4:05:11,  1.97it/s]

{'loss': 0.5673, 'grad_norm': 0.7912633419036865, 'learning_rate': 6.657789613848203e-06, 'epoch': 0.1}


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  5%|▍         | 1500/30033 [13:16<4:04:40,  1.94it/s]

...

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
100%|█████████▉| 30000/30033 [4:14:47<00:17,  1.94it/s]

{'loss': 0.4404, 'grad_norm': 2.6801083087921143, 'learning_rate': 2.4418217470124684e-08, 'epoch': 3.0}


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
100%|██████████| 30033/30033 [4:15:08<00:00,  1.96it/s]

{'train_runtime': 15308.6572, 'train_samples_per_second': 3.924, 'train_steps_per_second': 1.962, 'train_loss': 0.5135501870717479, 'epoch': 3.0}


TrainOutput(global_step=30033, training_loss=0.5135501870717479, metrics={'train_runtime': 15308.6572, 'train_samples_per_second': 3.924, 'train_steps_per_second': 1.962, 'total_flos': 2.891667335836877e+16, 'train_loss': 0.5135501870717479, 'epoch': 3.0})

In [83]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (rotary_emb): Qwen2RotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
                (quant_linear_modu

In [84]:
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

In [88]:
trainer.save_model('lora_qwen25_coder_7b')

#### 4. evaluate base model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
torch.cuda.empty_cache()

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('lora_qwen25_coder_7b')

In [3]:
from peft import PeftModel
model_name = 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8'
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)
base_model

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
`low_cpu_mem_usage` was None, now default

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151672, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (rotary_emb): Qwen2RotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): Qwen2MLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3584, out_features=151672, bias=False)
)

In [5]:
messages = [
    {"role": "system", "content": 'You are a useful assistant to generate python function by user instruction!'},
    {"role": "user", "content": 'Create a function implements fibonacci series in Python.'},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
text

'<|im_start|>system\nYou are a useful assistant to generate python function by user instruction!<|im_end|>\n<|im_start|>user\nCreate a function implements fibonacci series in Python.<|im_end|>\n<|im_start|>assistant\n'

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
base_model.to(device)
inputs = tokenizer(text, return_tensors='pt').to(device)
outputs = base_model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

system
You are a useful assistant to generate python function by user instruction!
user
Create a function implements fibonacci series in Python.
assistant
Certainly! Below is a simple Python function that generates the Fibonacci series up to a specified number of terms:

```python
def fibonacci_series(n):
    """
    Generate a list containing the first n numbers of the Fibonacci series.

    :param n: The number of terms in the Fibonacci series to generate.
    :return:


In [9]:
from transformers import pipeline
gen0 = pipeline('text-generation', model=base_model, tokenizer=tokenizer, device='cuda')
outputs = gen0(text, max_length=128)
print(outputs[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<|im_start|>system
You are a useful assistant to generate python function by user instruction!<|im_end|>
<|im_start|>user
Create a function implements fibonacci series in Python.<|im_end|>
<|im_start|>assistant
Certainly! Below is a simple Python function that generates the Fibonacci series up to a specified number of terms:

```python
def fibonacci(n):
    """
    Generate a list containing the first n numbers of the Fibonacci series.

    Parameters:
    n (int): The number of terms in the Fibonacci series to generate.

    Returns:
    list: A list containing the first n numbers of the Fibonacci series.
    """
    if n <= 0:
        return []



##### 4.1 clear cache

In [10]:
import gc
# del base_model
gc.collect()
torch.cuda.empty_cache()

#### 5. evaluate peft model

In [11]:
new_model = PeftModel.from_pretrained(base_model, 'lora_qwen25_coder_7b')
new_model.print_trainable_parameters()
new_model

trainable params: 0 || all params: 1,168,129,536 || trainable%: 0.0000


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151672, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (rotary_emb): Qwen2RotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
                (quant_linear_modu

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# new_model.to(device)
# input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
# output = new_model.generate(input_ids, max_length=64)
# print(tokenizer.decode(output[0], skip_special_tokens=False))

In [13]:
from transformers import pipeline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
generator = pipeline('text-generation', model=new_model, tokenizer=tokenizer, device=device)
print(generator(text, max_length=128)[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

<|im_start|>system
You are a useful assistant to generate python function by user instruction!<|im_end|>
<|im_start|>user
Create a function implements fibonacci series in Python.<|im_end|>
<|im_start|>assistant
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

for i in range(10):
    print(fibonacci(i))
